In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion, ball

# ==========================================
# CONFIG
# ==========================================
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_4 Airy-CBs_300425.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_0h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_0h_2_051222.czi"
# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

# Per-voxel metrics
voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale that preserves volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)

# LoG returns sigma; convert to a scale radius in index coords; for 3D use sqrt(3)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius in index units (pixels)
print(f"Detected {len(blobs)} lysosomes.")

# --- Convert per-lysosome metrics to physical units (µm) ---
if len(blobs) > 0:
    # positions in µm
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um

    # single radius/diameter/volume per lysosome (equivalent-sphere), in µm
    radius_um   = (blobs[:, 3] * lin_equiv_um)*0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

# --- Region bins (optional export kept) ---
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

# --- Per-blob DF (µm only) ---
blob_ids = np.arange(1, len(blobs) + 1)

df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
    #"region_id": region_labels
})
"""
if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_um", "size"),
        avg_diameter_um=("diameter_um", "mean"),
        total_volume_um3=("volume_um3", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter_um", "total_volume_um3"])
"""
# --- Save (µm-only) ---
df.to_csv("lysosome_blobs_regions.csv", index=False)
#region_summary.to_csv("lysosome_region_summary.csv", index=False)
#print("Saved: lysosome_blobs_regions.csv (µm-only) and lysosome_region_summary.csv (µm-only)")
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 3) Viewer base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE)
# ==========================================
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=91, offset=-0.3*np.std(R))#####0.4###81
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(7))#3
neuron_mask = binary_erosion(neuron_mask, ball(1))  # light extra erosion

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table (µm-only metrics)
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 5b) Per-cell (Ch2) volumes (already in µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 6) Visualization (CELL vs OUTSIDE only)
# ==========================================
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,  # size in index units for display
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and body_lab.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Save a 3D screenshot (post-segmentation)
# ==========================================

def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 9) Run viewer
# ==========================================
napari.run()

In [ ]:
###new version with original video mp4################3

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion, ball

# ==========================================
# CONFIG
# ==========================================
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_4 Airy-CBs_300425.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_0h_1_051222.czi"#comenze aca
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_0h_2_051222.czi"
# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

# Per-voxel metrics
voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale that preserves volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)

# LoG returns sigma; convert to a scale radius in index coords; for 3D use sqrt(3)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius in index units (pixels)
print(f"Detected {len(blobs)} lysosomes.")

# --- Convert per-lysosome metrics to physical units (µm) ---
if len(blobs) > 0:
    # positions in µm
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um

    # single radius/diameter/volume per lysosome (equivalent-sphere), in µm
    radius_um   = (blobs[:, 3] * lin_equiv_um)*0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

# --- Region bins (optional export kept) ---
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

# --- Per-blob DF (µm only) ---
blob_ids = np.arange(1, len(blobs) + 1)

df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
    #"region_id": region_labels
})
"""
if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_um", "size"),
        avg_diameter_um=("diameter_um", "mean"),
        total_volume_um3=("volume_um3", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter_um", "total_volume_um3"])
"""
# --- Save (µm-only) ---
df.to_csv("lysosome_blobs_regions.csv", index=False)
#region_summary.to_csv("lysosome_region_summary.csv", index=False)
#print("Saved: lysosome_blobs_regions.csv (µm-only) and lysosome_region_summary.csv (µm-only)")
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 3) Viewer base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE)
# ==========================================
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)#0.6

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=121, offset=-0.3*np.std(R))#91,0.3
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)#2000,3
neuron_mask = binary_closing(neuron_mask, ball(2))#7
neuron_mask = binary_erosion(neuron_mask, ball(1))#1

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
#cell_mask = binary_opening(cell_mask, ball(4))#3
cell_mask = binary_closing(cell_mask, ball(4))#4

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table (µm-only metrics)
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 5b) Per-cell (Ch2) volumes (already in µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 6) Visualization (CELL vs OUTSIDE only)
# ==========================================
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,  # size in index units for display
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and body_lab.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Save a 3D screenshot (post-segmentation)
# ==========================================

def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original .CZI as MP4
# ==========================================
try:
    # Normalize and prepare raw grayscale movie (channel 1 or 2)
    raw_stack = np.array(image, dtype=np.float32)
    raw_norm = (255 * (raw_stack - raw_stack.min()) / (raw_stack.ptp() + 1e-8)).astype(np.uint8)

    # Make a colored version for visualization
    frames_raw = []
    for z in range(raw_norm.shape[0]):
        frame_gray = raw_norm[z]
        frame_bgr = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        frames_raw.append(frame_bgr)

    # Save as MP4
    mp4_name = "original_czi_raw.mp4"
    imageio.mimsave(mp4_name, frames_raw, fps=8, format='FFMPEG')
    print(f"Saved: {mp4_name}")

except Exception as e:
    print("MP4 export of original CZI failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    # --- Helpers ---
    def to_uint8_grayscale(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return (np.zeros_like(vol, dtype=np.uint8))
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # Choose which channel you want on the LEFT as "raw"
    # (you can switch to image_2 if you prefer the neuron channel)
    raw_stack_u8 = to_uint8_grayscale(image)      # left panel (raw)
    seg_base_u8  = to_uint8_grayscale(image_2)    # base for the overlay (right)

    Z, H, W = raw_stack_u8.shape
    fps = 8

    # Stream-write to avoid big memory usage
    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=fps, format="FFMPEG")

    for z in range(Z):
        # ---------- LEFT: raw ----------
        left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)

        # ---------- RIGHT: overlay on Ch2 ----------
        base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)

        # cell mask overlay in green (same opacity you used above)
        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green channel
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        # draw lysosome circles (white if outside, yellow if inside cell)
        if 'blobs' in globals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W:
                    if 'cell_mask' in globals() and cell_mask[z, y, x]:
                        color = (255, 255, 0)  # yellow (inside cell)
                    else:
                        color = (255, 255, 255)  # white (outside)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

        # ---------- Match sizes & concat ----------
        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)

        # optional center divider
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)  # thin gray bar
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    # Fallback to GIF if ffmpeg is missing
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(raw_stack_u8.shape[0]):
            left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0:
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W:
                        color = (255, 255, 0) if ('cell_mask' in globals() and cell_mask[z, y, x]) else (255, 255, 255)
                        cv2.circle(right_bgr, (x, y), r, color, 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 11) Run viewer
# ==========================================
napari.run()